In [ ]:
# --- One cell: Downsample MIND-small (train users) and keep referenced news safely ---

import os, re, shutil, random
from typing import Set
import pandas as pd

# ==== Edit these paths/params ====
in_root  = "/baseline/data/original"
out_root = "/baseline/data_downsampled_20k/original"
seed = 42
train_users = 20000
val_users = -1   # set >0 if you also want to downsample val

# ==========================
# Helpers (no header games)
# ==========================
BEH_COLS = ["impression_id", "user_id", "time", "click_history", "impressions"]
SEP = re.compile(r"[,\s|]+")

def read_behaviors(path: str) -> pd.DataFrame:
    # behaviors.tsv in MIND has NO header and 5 columns
    return pd.read_csv(path, sep="\t", header=None, names=BEH_COLS, dtype=str, quoting=3)

def extract_news_from_behaviors(df: pd.DataFrame) -> Set[str]:
    """
    Collect news IDs from click_history and impressions.
    Handles space/comma/pipe separators and trims '-0/-1' labels.
    Keeps only IDs that look like MIND news IDs (start with 'N').
    """
    news_ids: Set[str] = set()

    # history
    for h in df["click_history"].fillna("").astype(str):
        if not h or h == "-":
            continue
        for tok in SEP.split(h.strip()):
            tok = tok.strip()
            if not tok:
                continue
            if "-" in tok:
                tok = tok.split("-", 1)[0]
            news_ids.add(tok)

    # impressions
    for imp in df["impressions"].fillna("").astype(str):
        if not imp:
            continue
        for tok in SEP.split(imp.strip()):
            tok = tok.strip()
            if not tok:
                continue
            if "-" in tok:
                tok = tok.split("-", 1)[0]
            news_ids.add(tok)

    return {nid for nid in news_ids if nid and nid[0] == "N"}

def sample_users(df: pd.DataFrame, target_users: int, seed: int) -> Set[str]:
    users = sorted(df["user_id"].astype(str).unique().tolist())
    if target_users <= 0 or target_users >= len(users):
        return set(users)
    rnd = random.Random(seed)
    return set(rnd.sample(users, target_users))

def subset_behaviors(df: pd.DataFrame, keep_users: Set[str]) -> pd.DataFrame:
    return df[df["user_id"].astype(str).isin(keep_users)].copy()

def filter_news_stream(news_path: str, keep_ids: Set[str], out_path: str):
    """
    Stream filter news.tsv by first column (true news_id).
    - Never writes a header
    - Preserves each kept line verbatim (so JSON columns remain valid)
    - Skips a header line automatically if the first token doesn't look like an 'N...' id.
    """
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    kept, total = 0, 0
    with open(news_path, "r", encoding="utf-8") as fin, open(out_path, "w", encoding="utf-8") as fout:
        first = True
        for line in fin:
            total += 1
            # news_id is the substring before the first tab
            tab = line.find("\t")
            token = line if tab == -1 else line[:tab]
            token = token.strip()

            if first:
                first = False
                # If first line is a header (e.g., 'news_id' / 'news' / 'category' etc.), skip it
                if not token.startswith("N"):
                    continue  # header-like -> drop
            # keep only if the first field is an ID we need
            if token in keep_ids:
                fout.write(line)
                kept += 1
    print(f"[news] {kept:,} rows kept out of ~{total:,} lines (train/news.tsv)")

def process_split(split_name: str, in_root: str, out_root: str, seed: int, target_users: int):
    in_split = os.path.join(in_root, split_name)
    out_split = os.path.join(out_root, split_name)
    os.makedirs(out_split, exist_ok=True)

    beh_in = os.path.join(in_split, "behaviors.tsv")
    news_in = os.path.join(in_split, "news.tsv")
    ent_in  = os.path.join(in_split, "entity_embedding.vec")
    rel_in  = os.path.join(in_split, "relation_embedding.vec")

    beh_out = os.path.join(out_split, "behaviors.tsv")
    news_out = os.path.join(out_split, "news.tsv")
    ent_out  = os.path.join(out_split, "entity_embedding.vec")
    rel_out  = os.path.join(out_split, "relation_embedding.vec")

    # Load & sample users
    beh_df = read_behaviors(beh_in)
    keep_users = sample_users(beh_df, target_users, seed)
    beh_sub = subset_behaviors(beh_df, keep_users)
    beh_sub.to_csv(beh_out, sep="\t", header=False, index=False)
    print(f"[{split_name}] behaviors written -> {beh_out} (rows: {len(beh_sub):,}, unique users: {beh_sub['user_id'].nunique():,})")

    # Get referenced news
    keep_ids = extract_news_from_behaviors(beh_sub)
    print(f"[{split_name}] referenced news ids: {len(keep_ids):,}")

    # Filter news.tsv by first column only, NO HEADER written
    print(f"[{split_name}] filtering news.tsv -> {news_out}")
    filter_news_stream(news_in, keep_ids, news_out)

    # Copy embeddings unchanged
    if os.path.exists(ent_in): shutil.copyfile(ent_in, ent_out)
    if os.path.exists(rel_in): shutil.copyfile(rel_in, rel_out)
    print(f"[{split_name}] Done.\n")

# -------------------- run --------------------
process_split("train/MINDsmall_train", in_root, out_root, seed, train_users)

if val_users == -1:
    print("[val] Copying validation split unchanged...")
    in_val = os.path.join(in_root, "val")
    out_val = os.path.join(out_root, "val")
    os.makedirs(out_val, exist_ok=True)
    for fname in ["behaviors.tsv", "news.tsv", "entity_embedding.vec", "relation_embedding.vec"]:
        src = os.path.join(in_val, fname)
        dst = os.path.join(out_val, fname)
        if os.path.exists(src):
            shutil.copyfile(src, dst)
    print("[val] Done (unchanged).")
else:
    process_split("val", in_root, out_root, seed, val_users)

print("All done!")


[train/MINDsmall_train] behaviors written -> /home/scur1695/ir2-colbert-news/baseline/data_downsampled_20k/original/train/MINDsmall_train/behaviors.tsv (rows: 62,668, unique users: 20,000)
[train/MINDsmall_train] referenced news ids: 38,411
[train/MINDsmall_train] filtering news.tsv -> /home/scur1695/ir2-colbert-news/baseline/data_downsampled_20k/original/train/MINDsmall_train/news.tsv
[news] 38,411 rows kept out of ~51,282 lines (train/news.tsv)
[train/MINDsmall_train] Done.

[val] Copying validation split unchanged...
[val] Done (unchanged).
All done!
